# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.


## Exploring the Dataset

We'll now start by reading in the dataset.

In [1]:
# Importing relevant libraries
import pandas as pd

In [2]:
# Reading in the data set
sms_df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label','SMS'])

# Checking the shape
sms_df.shape

(5572, 2)

In [3]:
# Printing first 5 rows
sms_df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Printing last 5 rows of the dataset
sms_df.tail()

,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [5]:
# Frequency table of 'Label' column
sms_df['Label'].value_counts(dropna=True, normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We can see that our `sms_df` DataFrame has 5572 rows and 2 columns. It labels `spam` messages as `spam` and `non-spam` messages as `ham`. The DataFrame has `86.6%` non-spam messages and `13.4%` spam messages.

## Training and Test Set

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [6]:
# Randomize the data
data_randomized = sms_df.sample(frac=1, random_state=1)

# Index for splitting
train_test_index = round(len(data_randomized) * 0.8)

# Splitting training and testing data
training = data_randomized[:train_test_index].reset_index(drop=True)
test = data_randomized[train_test_index:].reset_index(drop=True)

# Checking dimensions of split
print('Training data dimensions: ', training.shape)
print('Testing data dimensions: ', test.shape)

Training data dimensions:  (4458, 2)
Testing data dimensions:  (1114, 2)


In [7]:
training['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [8]:
test['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

We have split our original DataFrame into training and test DataFrames. Also, seeing the percentages of ham and spam in both `training` and `test` and verifying that it is approximately same as the original dataframe `sms_df`, we can safely go ahead.

## Data Cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

In [9]:
training.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
training['SMS'] = training['SMS'].str.replace('\W',' ').str.lower()
training.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [11]:
training['SMS'] = training['SMS'].str.split()

vocabulary = []

for sms in training['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

We're now going to use the vocabulary we just created to make the data transformation we want.

In [12]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [13]:
# Converting words_counts_per_sms dictionary to a dataframe
word_counts = pd.DataFrame(word_counts_per_sms)

# Print first 5 rows
word_counts.head()

,yeovil,4give,decision,outsider,weds,dudette,hesitate,tmorow,till,anytime,...,ramaduth,citylink,oyster,po,brought,sunny,mum,aah,payed2day,rather
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Joining both the dataframes 
training_clean = pd.concat([training, word_counts], axis=1)

# Printing first 5 rows of joined dataframe
training_clean.head()

,Label,SMS,yeovil,4give,decision,outsider,weds,dudette,hesitate,tmorow,...,ramaduth,citylink,oyster,po,brought,sunny,mum,aah,payed2day,rather
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [15]:
# Assigning 1 to alpha variable
alpha = 1

# Isolating spam and ham messages
spam_messages = training_clean[training_clean['Label'] == 'spam']
ham_messages = training_clean[training_clean['Label'] == 'ham']

# Number of words in vocabulary assigned
n_vocabulary = len(vocabulary)

# Calculating p_spam and p_ham
p_spam = len(spam_messages) / training_clean.shape[0]
p_ham = len(ham_messages) / training_clean.shape[0]

# Calculating number of words in spam messages and in ham messages
n_spam = spam_messages['SMS'].apply(len).sum()
n_ham = ham_messages['SMS'].apply(len).sum()

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [16]:
parameter_spam = {unique_word: 0 for unique_word in vocabulary}
parameter_ham = {unique_word: 0 for unique_word in vocabulary}

for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    parameter_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    parameter_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.
   

In [17]:
import re
def classify(message):
    
    sms_words = re.sub('\W',' ', message).lower().split()
    p_spam_given_words = p_spam
    p_ham_given_words = p_ham
    
    for word in sms_words:
        if word in parameter_spam:
            p_spam_given_words *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_words *= parameter_ham[word]
            
    print('P(spam|message):', p_spam_given_words)
    print('P(ham|message):', p_ham_given_words)
        
    if p_ham_given_words > p_spam_given_words:
        print('This message is a ham message.')
    elif p_ham_given_words < p_spam_given_words:
        print('This message is a spam message.')
    else:
        print('The message requires human help.')

In [18]:
# classify function trial
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(spam|message): 1.3481290211300841e-25
P(ham|message): 1.9368049028589875e-27
This message is a spam message.


In [19]:
# classify function trial
classify("Sounds good, Tom, then see u there")

P(spam|message): 2.4372375665888117e-25
P(ham|message): 3.687530435009238e-21
This message is a ham message.


## Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [20]:
def classify_test_set(message):
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]
            
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human help'

In [21]:
# Adding a new column 'predicted' with classification of messages
test['predicted'] = test['SMS'].apply(classify_test_set)

# Printing first 5 rows
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [22]:
# Checking the accuracy of our spam filter in test dataframe
accuracy_spam_filter = (test['predicted'] == test['Label']).sum() / test.shape[0]

print(accuracy_spam_filter * 100)

98.74326750448833


In [23]:
correct = (test['predicted'] == test['Label']).sum()
incorrect = test.shape[0] - correct

print('correct results: ', correct)
print('incorrect results: ', incorrect)

correct results:  1100
incorrect results:  14


We got an accuracy of 98.74% using our Spam filter algorithm, and this is satisfactory result as our algorithm got 1100 results correctly classified out of a total of 1114 results.

## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.